In [38]:
import torch
from torchvision import  datasets, transforms
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
import time

In [39]:
pipeline=transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.1307,),(0.3081,))]
)

In [40]:
from torch.utils.data import DataLoader

train_set=datasets.MNIST(
    root="../data",
    train=True,
    download=True,
    transform=pipeline
)

test_set=datasets.MNIST(
    root="../data",
    train=False,
    download=True,
    transform=pipeline
)

train_x=torch.unsqueeze(train_set.data, dim=1).type(torch.float32)
train_x=train_x.view(-1,28*28)
train_y=train_set.targets

test_x=torch.unsqueeze(test_set.data, dim=1).type(torch.float32)
test_x=test_x.view(-1,28*28)
test_y=test_set.targets

trainData=np.array(train_x)
trainLabels=np.array(train_y)
testData=np.array(test_x)
testLabels=np.array(test_y)

valData = testData
valLabels = testLabels

In [41]:
# print(trainData.shape), print(trainLabels.shape)
# print(testData.shape), print(testLabels.shape)
# print(valData.shape), print(valLabels.shape)

In [42]:
kVals = range(1, 30, 1)
accuracies = []

# loop over kVals
for k in range(1, 30, 1):
    # train the classifier with the current value of `k`
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(trainData, trainLabels)
    # evaluate the model and print the accuracies list
    score = model.score(valData, valLabels)
    with open("../result/KNN.txt", "a") as f:
        f.write("k=%d, accuracy=%.2f%%\n" % (k, score * 100))
    accuracies.append(score)

# largest accuracy
# np.argmax returns the indices of the maximum values along an axis
i = np.argmax(accuracies)
with open("../result/KNN.txt", "a") as f:
    f.write("k=%d achieved highest accuracy of %.2f%% on validation data\n" % (kVals[i], accuracies[i] * 100))

# Now that I know the best value of k, re-train the classifier
model = KNeighborsClassifier(n_neighbors=kVals[i])
start=time.time()
model.fit(trainData, trainLabels)
# Predict labels for the test set
end=time.time()
with open("../result/KNN.txt", "a") as f:
    f.write("Training time: %.2f seconds\n" % (end-start))
start=time.time()
predictions = model.predict(testData)
end=time.time()
with open("../result/KNN.txt", "a") as f:
    f.write("Testing time: %.2f seconds\n" % (end-start))
# Evaluate performance of model for each of the digits
with open("../result/KNN.txt", "a") as f:
    f.write("EVALUATION ON TESTING DATA\n")
    f.write(classification_report(testLabels, predictions))
